In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from warnings import simplefilter
os.chdir("../")
from src.data import import_data, MetadataStats
from src.features.build_features import MetaDataManagement, DataManagement

In [2]:
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.set_option('display.max_columns', None)

In [ ]:
model_data = pickle.load(
    open('./data/app_inputs/modeling_data.pkl', 'rb'))

rdmf = pickle.load(
    open('./data/models/rdmf_model.pkl', 'rb'))

In [ ]:
rdmf.score(model_data.X_valid_sparse, model_data.y_valid)

In [ ]:
rdmf.score(model_data.X_train_sparse, model_data.y_train)

In [3]:
df = import_data()
# MetaDataManagement(df).metadata_management_pipeline()
# sp_mat, cols = DataManagement(df).data_management_pipeline()

# Idea Testing

In [4]:
comms_df = df.copy()

In [ ]:
comms_df.avg_arpu_3m.mode()

In [ ]:
MetadataStats(comms_df).variable_summarizing("call_category_1")

In [ ]:
MetadataStats(comms_df).metadata_report('char')

In [ ]:
df_ = comms_df["verbatims"].to_frame()
df_["verbatims"] = df_["verbatims"].apply(lambda x: x.decode("utf-8"))
df_["verbatims_process"] = df_["verbatims"].copy()

In [ ]:
df_.head()

In [ ]:
comms_df.shape

In [ ]:
docs = df_["verbatims_process"].to_list()

In [ ]:
"""
from bertopic import BERTopic, vectorizers
vectorizer_model = CountVectorizer(stop_words="english")
ctfidf_model = vectorizers.ClassTfidfTransformer(reduce_frequent_words=True)

topic_model = BERTopic(
    embedding_model="bert-base-cased",
    # ctfidf_model=ctfidf_model,
    n_gram_range=(1, 3))

topics, probs = topic_model.fit_transform(docs)
topic_model.reduce_topics(docs, nr_topics=15)"""

In [ ]:
"""data = import_from_local(".")
MetaDataManagement(data).metadata_management_pipeline()
st.dataframe(data.head())

st.markdown("# Categorical features processing")


### splitting
X_train, X_valid, y_train, y_valid = train_valid_split(data)
    
## data variable management pipeline
DataManagement(X_train).data_management_pipeline()
DataManagement(X_valid).data_management_pipeline()


# models pipelines
## random forest model
evaluate_rdmf(X_train, X_valid, y_train, y_valid, n_estimators=20)

# model comparaison"""